In [ ]:
import os
import sys
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import scipy.stats

import matplotlib.pyplot as plt


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_visible_devices([gpus[1]], 'GPU')

sys.path.append("/nfs/scistore12/chlgrp/vvolhejn/smooth")

os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/0221_gp2/")

In [ ]:
df = pd.read_feather("measures.feather")
smooth.analysis.remove_constant_columns(df)
df["log_dir"] = df["log_dir"].str.split("/").str.get(-1)
df

In [ ]:
%load_ext autoreload
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

In [ ]:
for col in df.columns:
    if df[col].dtype == "object":
        continue
    plt.hist(df[col], bins=20)
    plt.title(col)
    plt.show()

In [ ]:
dataset = smooth.datasets.GaussianProcessDataset(
    x_min=-2, x_max=2, samples_train=10, samples_test=100, seed=123
)

for i, row in df.sort_values("train_loss").iterrows():
    log_dir = row["log_dir"]
    model = tf.keras.models.load_model(os.path.join(log_dir, "model.h5"))
    smooth.analysis.plot_shallow(model, dataset, title=log_dir)

In [ ]:
plt.plot(dataset.x_test, dataset.y_test)

In [ ]:
ken = smooth.analysis.get_kendall_coefs(
    df,
    ["batch_size", "hidden_size", "init_scale", "learning_rate", "iteration"],
    "test_accuracy",
    ["accuracy", "actual_epochs", "gradient_norm", "l2", "loss", "test_loss",
     "seg_total_variation", "seg_total_variation_derivative"]
)

In [ ]:
ken.astype("float32").round(4)
#.style.background_gradient(axis=None)

In [ ]:
corr = df.corr().round(2)
corr.style.background_gradient(axis=None)